# Part 1

--- Day 17: Conway Cubes ---

As your flight slowly drifts through the sky, the Elves at the Mythical Information Bureau at the North Pole contact you. They'd like some help debugging a malfunctioning experimental energy source aboard one of their super-secret imaging satellites.

The experimental energy source is based on cutting-edge technology: a set of Conway Cubes contained in a pocket dimension! When you hear it's having problems, you can't help but agree to take a look.

The pocket dimension contains an infinite 3-dimensional grid. At every integer 3-dimensional coordinate (`x,y,z`), there exists a single cube which is either active or inactive.

In the initial state of the pocket dimension, almost all cubes start inactive. The only exception to this is a small flat region of cubes (your puzzle input); the cubes in this region start in the specified active (`#`) or inactive (`.`) state.

The energy source then proceeds to boot up by executing six cycles.

Each cube only ever considers its neighbors: any of the 26 other cubes where any of their coordinates differ by at most 1. For example, given the cube at `x=1,y=2,z=3`, its neighbors include the cube at `x=2,y=2,z=2`, the cube at `x=0,y=2,z=3`, and so on.

During a cycle, all cubes simultaneously change their state according to the following rules:

- If a cube is active and exactly 2 or 3 of its neighbors are also active, the cube remains active. Otherwise, the cube becomes inactive.
- If a cube is inactive but exactly 3 of its neighbors are active, the cube becomes active. Otherwise, the cube remains inactive.

The engineers responsible for this experimental energy source would like you to simulate the pocket dimension and determine what the configuration of cubes should be at the end of the six-cycle boot process.

For example, consider the following initial state:

```
.#.
..#
###
```

Even though the pocket dimension is 3-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a `3x3x1` region of the 3-dimensional space.)

Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given z coordinate (and the frame of view follows the active cells in each cycle):

Before any cycles:

```
z=0
.#.
..#
###
```

After 1 cycle:

```
z=-1
#..
..#
.#.

z=0
#.#
.##
.#.

z=1
#..
..#
.#.
```

After 2 cycles:

```
z=-2
.....
.....
..#..
.....
.....

z=-1
..#..
.#..#
....#
.#...
.....

z=0
##...
##...
#....
....#
.###.

z=1
..#..
.#..#
....#
.#...
.....

z=2
.....
.....
..#..
.....
.....
```

After 3 cycles:

```
z=-2
.......
.......
..##...
..###..
.......
.......
.......

z=-1
..#....
...#...
#......
.....##
.#...#.
..#.#..
...#...

z=0
...#...
.......
#......
.......
.....##
.##.#..
...#...

z=1
..#....
...#...
#......
.....##
.#...#.
..#.#..
...#...

z=2
.......
.......
..##...
..###..
.......
.......
.......
```

After the full six-cycle boot process completes, 112 cubes are left in the active state.

Starting with your given initial configuration, simulate six cycles. How many cubes are left in the active state after the sixth cycle?

In [6]:
(setv MY-INPUT #[[
.###..#.
##.##...
....#.#.
#..#.###
...#...#
##.#...#
#..##.##
#.......]])

In [7]:
(setv TEST-INPUT #[[
.#.
..#
###]])

In [18]:
(defn load-input [str_]
  "convert input string grid to dict form"
  ; 0: inactive, 1: active
  ; top-left is (0 0 0), x and y increasing down and to the right
  (setv space {})
  (setv lines (str_.split))
  (for [row (range (len lines))]
    (for [col (-> (get lines 0) (len) (range))]
      (assoc space (, row col 0) (if (= (-> (get lines row) (get col)) "#") 1 0))))
  space)

In [19]:
(load-input TEST-INPUT)

{(0, 0, 0): 0, (0, 1, 0): 1, (0, 2, 0): 0, (1, 0, 0): 0, (1, 1, 0): 0, (1, 2, 0): 1, (2, 0, 0): 1, (2, 1, 0): 1, (2, 2, 0): 1}

In [14]:
(setv ADJACENCY (tuple (gfor nums (product (range -1 2) :repeat 3) :if (!= nums (, 0 0 0)) nums)))

In [17]:
(defn adjacent-coords [coords]
  (for [adj ADJACENCY] (yield (tuple (map sum (zip coords adj))))))

In [20]:
(defn adjacent-values [space coords]
  (for [adj (adjacent-coords coords)] (yield (space.get adj 0))))

In [24]:
(defn extent [coords]
  "given coords, return ranges that cover a space enclosing coords -1/+1"
  (setv [min-x min-y min-z] (* [1000] 3))
  (setv [max-x max-y max-z] (* [-1000] 3))
  (for [[x y z] coords]
    (when (< x min-x) (setv min-x x))
    (when (> x max-x) (setv max-x x))
    (when (< y min-y) (setv min-y y))
    (when (> y max-y) (setv max-y y))
    (when (< z min-z) (setv min-z z))
    (when (> z max-z) (setv max-z z)))
  (,
    (range (dec min-x) (+ max-x 2))
    (range (dec min-y) (+ max-y 2))
    (range (dec min-z) (+ max-z 2))))

In [25]:
(extent [[0 0 0]])

(range(-1, 2), range(-1, 2), range(-1, 2))

In [31]:
(defn do-cycle [space]
  (setv space-prime {})
  (for [coords
        (product #* (extent (filter (fn [coords] (get space coords)) (space.keys))))]
    (setv current-state (space.get coords 0))
    (setv neighbors-active (sum (adjacent-values space coords)))
    (assoc space-prime coords
      (cond
        [(and (= current-state 1) (< 1 neighbors-active 4)) 1]
        [(and (= current-state 0) (= neighbors-active 3)) 1]
        [True 0])))
  space-prime)

In [35]:
(defn cycle-for [space iterations]
  (for [_ (range iterations)] (setv space (do-cycle space)))
  space)

In [38]:
(sum (.values (cycle-for (load-input TEST-INPUT) 6)))

112

In [39]:
(sum (.values (cycle-for (load-input MY-INPUT) 6)))

304

# Part 2

For some reason, your simulated results don't match what the experimental energy source engineers expected. Apparently, the pocket dimension actually has four spatial dimensions, not three.

The pocket dimension contains an infinite 4-dimensional grid. At every integer 4-dimensional coordinate (`x,y,z,w`), there exists a single cube (really, a hypercube) which is still either active or inactive.

Each cube only ever considers its neighbors: any of the 80 other cubes where any of their coordinates differ by at most 1. For example, given the cube at `x=1,y=2,z=3,w=4`, its neighbors include the cube at `x=2,y=2,z=3,w=3`, the cube at `x=0,y=2,z=3,w=4`, and so on.

The initial state of the pocket dimension still consists of a small flat region of cubes. Furthermore, the same rules for cycle updating still apply: during each cycle, consider the number of active neighbors of each cube.

For example, consider the same initial state as in the example above. Even though the pocket dimension is 4-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a `3x3x1x1` region of the 4-dimensional space.)

Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given z and w coordinate:

Before any cycles:

```
z=0, w=0
.#.
..#
###
```

After 1 cycle:

```
z=-1, w=-1
#..
..#
.#.

z=0, w=-1
#..
..#
.#.

z=1, w=-1
#..
..#
.#.

z=-1, w=0
#..
..#
.#.

z=0, w=0
#.#
.##
.#.

z=1, w=0
#..
..#
.#.

z=-1, w=1
#..
..#
.#.

z=0, w=1
#..
..#
.#.

z=1, w=1
#..
..#
.#.
```

After 2 cycles:

```
z=-2, w=-2
.....
.....
..#..
.....
.....

z=-1, w=-2
.....
.....
.....
.....
.....

z=0, w=-2
###..
##.##
#...#
.#..#
.###.

z=1, w=-2
.....
.....
.....
.....
.....

z=2, w=-2
.....
.....
..#..
.....
.....

z=-2, w=-1
.....
.....
.....
.....
.....

z=-1, w=-1
.....
.....
.....
.....
.....

z=0, w=-1
.....
.....
.....
.....
.....

z=1, w=-1
.....
.....
.....
.....
.....

z=2, w=-1
.....
.....
.....
.....
.....

z=-2, w=0
###..
##.##
#...#
.#..#
.###.

z=-1, w=0
.....
.....
.....
.....
.....

z=0, w=0
.....
.....
.....
.....
.....

z=1, w=0
.....
.....
.....
.....
.....

z=2, w=0
###..
##.##
#...#
.#..#
.###.

z=-2, w=1
.....
.....
.....
.....
.....

z=-1, w=1
.....
.....
.....
.....
.....

z=0, w=1
.....
.....
.....
.....
.....

z=1, w=1
.....
.....
.....
.....
.....

z=2, w=1
.....
.....
.....
.....
.....

z=-2, w=2
.....
.....
..#..
.....
.....

z=-1, w=2
.....
.....
.....
.....
.....

z=0, w=2
###..
##.##
#...#
.#..#
.###.

z=1, w=2
.....
.....
.....
.....
.....

z=2, w=2
.....
.....
..#..
.....
.....
```

After the full six-cycle boot process completes, 848 cubes are left in the active state.

Starting with your given initial configuration, simulate six cycles in a 4-dimensional space. How many cubes are left in the active state after the sixth cycle?

In [40]:
(defn load-input2 [str_]
  "convert input string grid to dict form"
  ; 0: inactive, 1: active
  ; top-left is (0 0 0 0), x and y increasing down and to the right
  (setv space {})
  (setv lines (str_.split))
  (for [row (range (len lines))]
    (for [col (-> (get lines 0) (len) (range))]
      (assoc space (, row col 0 0) (if (= (-> (get lines row) (get col)) "#") 1 0))))
  space)

In [41]:
(setv ADJACENCY2 (tuple (gfor nums (product (range -1 2) :repeat 4) :if (!= nums (, 0 0 0 0)) nums)))

In [44]:
(defn adjacent-coords2 [coords adjacency]
  (for [adj adjacency] (yield (tuple (map sum (zip coords adj))))))

In [52]:
(defn adjacent-values2 [space coords adjacency]
  (for [adj (adjacent-coords2 coords adjacency)] (yield (space.get adj 0))))

In [50]:
(defn extent2 [coords]
  "given coords, return ranges that cover a space enclosing coords -1/+1"
  (setv [min-x min-y min-z min-w] (* [1000000] 4))
  (setv [max-x max-y max-z max-w] (* [-1000000] 4))
  (for [[x y z w] coords]
    (when (< x min-x) (setv min-x x))
    (when (> x max-x) (setv max-x x))
    (when (< y min-y) (setv min-y y))
    (when (> y max-y) (setv max-y y))
    (when (< z min-z) (setv min-z z))
    (when (> z max-z) (setv max-z z))
    (when (< w min-w) (setv min-w w))
    (when (> w max-w) (setv max-w w)))
  (,
    (range (dec min-x) (+ max-x 2))
    (range (dec min-y) (+ max-y 2))
    (range (dec min-z) (+ max-z 2))
    (range (dec min-w) (+ max-w 2))))

In [46]:
(defn do-cycle2 [space]
  (setv space-prime {})
  (for [coords
        (product #* (extent2 (filter (fn [coords] (get space coords)) (space.keys))))]
    (setv current-state (space.get coords 0))
    (setv neighbors-active (sum (adjacent-values2 space coords ADJACENCY2)))
    (assoc space-prime coords
      (cond
        [(and (= current-state 1) (< 1 neighbors-active 4)) 1]
        [(and (= current-state 0) (= neighbors-active 3)) 1]
        [True 0])))
  space-prime)

In [47]:
(defn cycle-for2 [space iterations]
  (for [_ (range iterations)] (setv space (do-cycle2 space)))
  space)

In [53]:
(sum (.values (cycle-for2 (load-input2 TEST-INPUT) 6)))

848

In [54]:
(sum (.values (cycle-for2 (load-input2 MY-INPUT) 6)))

1868